<a href="https://colab.research.google.com/github/Hemanth-stack/kaggle_competitions/blob/main/final_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import pandas as pd

data = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
passenger_id = test['PassengerId']
def clean(data):
  columns = ['PassengerId','Name','Ticket']
  for col in columns:
    data = data.drop(col,axis=1)

  data['Age'].fillna(value=data['Age'].median(),inplace=True)
  data['Cabin'].fillna(value='A',inplace=True)
  data['Embarked'].fillna(value='x',inplace=True)
  data['Fare'].fillna(value=data['Fare'].mean(),inplace=True)

  return data
data=clean(data)

test = clean(test)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,A,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,A,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,A,S


In [109]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.5,0,0,7.8292,A,Q
1,3,female,47.0,1,0,7.0000,A,S
2,2,male,62.0,0,0,9.6875,A,Q
3,3,male,27.0,0,0,8.6625,A,S
4,3,female,22.0,1,1,12.2875,A,S


In [110]:

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
columns_1 = ['Sex','Embarked']
for col in columns_1:
  data[col] = le.fit_transform(data[col])
  test[col] = le.transform(test[col])
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,34.5,0,0,7.8292,A,1
1,3,0,47.0,1,0,7.0000,A,2
2,2,1,62.0,0,0,9.6875,A,1
3,3,1,27.0,0,0,8.6625,A,2
4,3,0,22.0,1,1,12.2875,A,2


In [111]:
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_absolute_error

features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

X = data[features]
y = data['Survived']
train_X,val_X,train_y,val_y = train_test_split(X,y,random_state = 1)
d = dict()
for i in range(10,500):
  model = DTR(max_leaf_nodes=i)
  model.fit(train_X,train_y)
  predictions = model.predict(val_X)
  predictions = list(map(int,predictions))
  d[mean_absolute_error(val_y,predictions)]=i

#print(predictions)
print(d[min(d.keys())])
final_model = DTR(max_leaf_nodes=d[min(d.keys())])
final_model.fit(X,y)
predictions = final_model.predict(val_X)
predictions = list(map(int,predictions))
print(accuracy_score(val_y,predictions))

447
0.9417040358744395


In [115]:
test_X = test[features]
test_X.isnull().value_counts()
test_predictions = list(map(int,final_model.predict(test_X)))

In [116]:
output = pd.DataFrame({'PassengerId':passenger_id,"Survived":test_predictions})
output.to_csv("submission.csv",index=False)